In [104]:
import networkx as nx
import numpy as np
from tkinter import *
from tkinter import ttk
from PIL import ImageTk,Image
import matplotlib
import matplotlib.pylab as plt
import copy as cp
import os
import random

## Possible things in our environment

class Thing():
    def __init__(self, mark):
        self.mark = mark
    def display(self):
        return self.mark
    def setMark(self, newMark):
        self.mark = newMark
    def getMark(self):
        return self.mark


class Environment(Thing):
    def __init__(self, mark, color):
        self.mark = mark
        self.color = color
    def getColor(self):
        return self.color
    
class Grass(Environment):
    def __init__(self):
        super().__init__(',', 'DarkOliveGreen3')
class Dirt(Environment):
    def __init__(self):
        super().__init__('.', 'bisque3')


class Animal(Thing):
    def __init__(self, pos, age, ID,persona1=[0.33,0.33,0.34],stats1=[1,1,.5,.25]):
        super().__init__('a')
        self.age = age
        self.mark = 'a'
        self.mem = None
        self.img = None
        self.population = None
        self.position = pos
        self.setMark(ID)
        #grooming%,fight%,nuetral %
        self.persona=persona1
        #speed,vison,fight_a,courtship%
        self.stats=stats1
        self.food=100
        z=1
        rand=random.uniform(0,1)
        if(rand>.5):
            z=0
        self.sex=z
        #0=food_avoid_encounters, 1=food_find_encounters 2=food_nuetral_encounters,3=find_mate 
        a=0
        if(rand<persona1[0]):
            a=1
        elif(rand<(persona1[0]+persona1[1])):
            a=2
        self.state=a
    def getVision(self, myMap):
        priorityList = list()
        for i in range(self.attributes["Vision"]):
            if i+self.position[0]<myMap.getWidth():
                priorityList.append(myMap.getAt())
    def setPos(self,pos):
        self.position = pos
    def getPos(self):
        return self.position
    def getPop(self):
        return self.population
    
    def actions(self, arr): # vision? more than one?
        state = self.position
        l = len(arr)-1
        
        actions = []
        if 0<=state[0]<l:
            actions.append('S')
        if 0<state[0]<=l:
            actions.append('N')
        if 0<=state[1]<l:
            actions.append('E')
        if 0<state[1]<=l:
            actions.append('W')
        actions.append('X') # stay in place
        return actions
    
    def result(self, action):
        state = self.position
        def switch_act(action):
            switcher = {
                'N':(state[0]-1,state[1]),
                'S':(state[0]+1,state[1]),
                'E':(state[0],state[1]+1), 
                'W':(state[0],state[1]-1),
                'X':(state[0],state[1])
            }
            return switcher.get(action,"ERROR")
        return switch_act(action)
    
    def move(self, arr):
        action_list = self.actions(arr)
        action = random.choice(action_list)
        self.setPos(self.result(action))
        return self.getPos()
    #0=food_avoid_encounters, 1=food_find_encounters 2=food_nuetral_encounters,3=find_mate 
    def where_to(self,arr):
        if(self.state==0):
            action_list = self.actions(arr)
            for i in range(len(action_list)):
                if(len(arr[i])>0):
                    action_list.pop(i)-
                elif(arr[i][0]==0):
                    action_list.pop(i)
            if(len(action_list)==0):
                action_list = self.actions(arr)
                for i in range(len(action_list)):
                    if(len(arr[i])>0):
                        action_list.pop(i)
                action = random.choice(action_list)
                self.setPos(self.result(action))
                return self.getPos()
            
            action = random.choice(action_list)
            self.setPos(self.result(action))
            return self.getPos()
            print("food_avoid_encounters")
        elif(self.state==1):
            print("food_find_encounters")
        elif(self.state==2):
            print("food_nuetral_encounters")
        else:
            print("find_mate")
    
class Lion(Animal):
    def __init__(self, pos, age, ID):
        super().__init__(pos, age, ID)
        self.img = None
        self.population = .2 # percentage of map covered

class Interaction(Thing):
    def __init__(self, pos, listAnimals):
        chance = random.uniform(0,1)
        if len(listAnimals)==2:
            if chance<0.5:
                super().__init__('x') # fight - negative
            else:
                super().__init__('o') # greeting - positive
        else:
            super().__init__('g') # group - neither
            
        self.position = pos
        self.animalList = listAnimals
        self.num = len(listAnimals)
    def getPos(self):
        return self.position
    def getAnimals(self):
        return self.animalList
        

In [105]:
## Setting up our environement
import random

class Map():
    def __init__(self, size): # multiple things can be in one spot in map: ex. grass and animal
        #arrList = [[list([Grass(),]),list([Grass(),]), list([Dirt(),])],
        #           [list([Grass(),]),list([Grass(),]),list([Animal((1,1), 0, None, None),Dirt(),])],
        #           [list([Grass(),]),list([Grass(),]), list([Dirt(),])]]
        self.array = None
        self.animals = dict()
        self.G = nx.Graph()
        self.randomMap(size)
    def getG(self):
        return self.G
    def getArray(self):
        return self.array
    def getAnimals(self):
        return self.animals
    def getWidth(self):
        return len(self.array)
    def getHeight(self):
          return len(self.array[0]) # assumes square structre
    def getAt(self,pos):
        return self.array[pos[0]][pos[1]]
    def removeAnimal(self, pos, ID):
        for i in self.array[pos[0]][pos[1]]:
            if isinstance(i, Animal):
                if i.getMark() == ID:
                    self.array[pos[0]][pos[1]].remove(i)
    def placeAnimal(self,a):
        self.array[a.position[0]][a.position[1]].insert(0, a)
    def randomMap(self, size):
        arrTemp = list()
        for i in range(size):
            col = list()
            for j in range(size):
                itemList = list()
                chance = random.uniform(0,1)
                for cls in Animal.__subclasses__():
                    age = random.uniform(1,10)
                    ID = str(random.randint(0, 100000))
                    subClass = cls((i,j), age, ID)
                    if chance < subClass.getPop():
                        itemList.append(subClass)
                        self.animals[ID] = subClass
                        self.G.add_node(str(ID))
                if chance <= 0.5:
                    itemList.append(Grass())
                else:
                    itemList.append(Dirt())
                col.append(itemList)
            arrTemp.append(col)
        self.array = arrTemp
    
    def organizeList(self, pos):
        if len(self.array[pos[0]][pos[1]]) > 1:
            pass # implement order Animal -> Grass
        else:
            pass
    def displayPos(self, pos):
        self.organizeList(pos)
        self.array[pos[0]][pos[1]].display()

    def display(self):
        for i in range(len(self.array)):
            for j in range(len(self.array[0])):
                print(self.array[i][j][0].display()+" ", end="") # print first item in list
            print("")

In [106]:
myMap = Map(6)
myMap.display()

. 95031 6296 , 70156 . 
. 37324 9596 . 2112 . 
, . . . 63202 . 
65781 . . 41248 . . 
. 80771 , 70349 24776 66089 
. . . . , . 


In [107]:
from PIL import Image, ImageTk


global myMap
global entry



def setForm(root):    
    global myMap
    cellrelheight= 1/myMap.getHeight()*(270/300) # 270 = 300 - (300*0.1)
    cellrelwidth=1/myMap.getWidth()
    
    for r in range(myMap.getHeight()):
        for c in range(myMap.getWidth()):
            bg = 'white'
            thingList = myMap.getAt((r,c))
            img = None
            text = None
            for i in thingList:
                if isinstance(i, Environment):
                    bg = i.color
                if isinstance(i, Animal):
                    text = i.mark
                    myMap.getG().add_node(text)
            
            relx = (cellrelwidth*c/2)
            rely = 0.1 + (cellrelheight*r)
            b = Label(root, bg=bg, image=img, text=text, borderwidth=3,relief="sunken")
            b.place(relx=relx, rely=rely, relwidth=cellrelwidth/2, relheight=cellrelheight)
    
    if os.path.exists("envGraph.jpg"):
        os.remove("envGraph.jpg")
        
    nx.draw_networkx(myMap.getG())
    limits=plt.axis('off')                                                                # turn off axes
    plt.savefig("envGraph.jpg")
    drawG(root,"envGraph.jpg", cellrelwidth)
    
def changeForm(root, listInteractions):
    global myMap
    cellrelheight= 1/myMap.getHeight()*(270/300) # 270 = 300 - (300*0.1)
    cellrelwidth=1/myMap.getWidth() # should make global?
    
    for r in range(myMap.getHeight()):
        for c in range(myMap.getWidth()):
            bg = 'white'
            thingList = myMap.getAt((r,c))
            img = None
            text = None
            for i in thingList:
                if isinstance(i, Environment):
                    bg = i.color
                    
                flagInteraction = False
                for interaction in listInteractions:
                    if (r,c) == interaction.getPos():
                        flagInteraction = True
                        text = interaction.getMark()
                        for a in interaction.getAnimals():
                            for a2 in interaction.getAnimals():
                                if a != a2:
                                    myMap.getG().add_edge(a.mark, a2.mark) # need to check type
                                    # for different kinds of edges
                if isinstance(i, Animal) and flagInteraction==False:
                    text = i.mark
                    G.add_node(text)
                
            relx = (cellrelwidth*c/2)
            rely = 0.1 + (cellrelheight*r)
            b = Label(root, bg=bg, image=img, text=text, borderwidth=3,relief="sunken")
            b.place(relx=relx, rely=rely, relwidth=cellrelwidth/2, relheight=cellrelheight)
    
    if len(listInteractions)>0:
        nx.draw_networkx(myMap.getG())
        limits=plt.axis('off')                                                                # turn off axes
        plt.savefig("envGraph.jpg")
        drawG(root,"envGraph.jpg", cellrelwidth)

def drawG(root, path, cellrelwidth):
    img= (Image.open(path))
    im = img.resize((int(290), int(320)))
    img_r = ImageTk.PhotoImage(im)
    
    panel = ttk.Label(root, image = img_r)
    panel.place(relx=0.5, rely=0.1, relwidth=0.5, relheight=1)
    plt.clf()
    root.mainloop()
    
def simulate():
    root = Tk()
    root.wm_title("Network Final")
    root.geometry("600x330")
    frm = ttk.Frame(root)

    separator = ttk.Separator(root, orient='vertical')
    separator.place(relx=0.5, rely=0, relwidth=0.01, relheight=1)
    
    separator = ttk.Separator(root, orient='horizontal')
    separator.place(relx=0, rely=0.1, relwidth=1, relheight=0.01)
    
    entry = Entry(root)
    entry.place(relx=0.1, rely=0, relwidth=0.05, relheight=0.1)
    entry.focus_set()
    
    def getSize():
        size = int(entry.get())
        global myMap
        myMap = Map(size)
        setForm(root)
        
    def timeStep():
        global myMap
        if myMap is not None:
            temp = cp.deepcopy(myMap.getArray())
            for a in myMap.getAnimals().values():
                oldPos = a.getPos()
                a.move(temp)
                myMap.removeAnimal(oldPos, a.getMark()) 
                myMap.placeAnimal(a)
            
            listInteractions = list()
            interactionPos = list()
            for a in myMap.getAnimals().values():
                if a.getPos() not in interactionPos:
                    interactionPos.append(a.getPos())
                    things = myMap.getAt(a.getPos())
                    animalCount = 0
                    animalList = list()
                    for t in things:
                        if isinstance(t, Animal):
                            animalCount += 1
                            animalList.append(t)
                    if animalCount>=2:
                        listInteractions.append(Interaction(a.getPos(), animalList))
                        
            changeForm(root, listInteractions)
        else:
            print("No map")
            
        
    
    buttonSize = Button(root, text= "Set Map:",command=getSize, bg="ivory3")
    buttonSize.place(relx=0, rely=0, relwidth=0.1, relheight=0.1)
    
    time_button = Button(root, text="Timestep", command=timeStep, bg="ivory3")
    time_button.place(relx=0.5, rely=0, relwidth=0.5, relheight=0.1)
    
    root.mainloop()

In [109]:
simulate()

<Figure size 432x288 with 0 Axes>